In [0]:
!pip3 -qq install torch==0.4.1
!pip install -qq bokeh==0.13.0
!pip install -qq gensim==3.6.0

In [0]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


if torch.cuda.is_available():
    from torch.cuda import FloatTensor, LongTensor
else:
    from torch import FloatTensor, LongTensor

np.random.seed(42)

# Рекуррентные нейронные сети, часть 2

## POS Tagging

Мы уже посмотрели на применение рекуррентных сетей для классификации.

![RNN types](http://karpathy.github.io/assets/rnn/diags.jpeg =x250)

*From [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)*

Перейдем к ещё одному варианту - sequence labeling (последняя картинка).

Самые популярные примеры для такой постановки задачи - Part-of-Speech Tagging и Named Entity Recognition.

Мы порешаем сейчас POS Tagging для английского.

Будем работать с таким набором тегов:
- ADJ - adjective (new, good, high, ...)
- ADP - adposition (on, of, at, ...)
- ADV - adverb (really, already, still, ...)
- CONJ - conjunction (and, or, but, ...)
- DET - determiner, article (the, a, some, ...)
- NOUN - noun (year, home, costs, ...)
- NUM - numeral (twenty-four, fourth, 1991, ...)
- PRT - particle (at, on, out, ...)
- PRON - pronoun (he, their, her, ...)
- VERB - verb (is, say, told, ...)
- . - punctuation marks (. , ;)
- X - other (ersatz, esprit, dunno, ...)

Скачаем данные:

In [0]:
import nltk
from sklearn.cross_validation import train_test_split

nltk.download('brown')
nltk.download('universal_tagset')

data = nltk.corpus.brown.tagged_sents(tagset='universal')

Пример размеченного предложения:

In [0]:
for word, tag in data[0]:
    print('{:15}\t{}'.format(word, tag))

Построим разбиение на train/val/test - наконец-то, всё как у нормальных людей.

На train будем учиться, по val - подбирать параметры и делать всякие early stopping, а на test - принимать модель по ее финальному качеству.

In [0]:
train_data, test_data = train_test_split(data, test_size=0.25, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.15, random_state=42)

print('Words count in train set:', sum(len(sent) for sent in train_data))
print('Words count in val set:', sum(len(sent) for sent in val_data))
print('Words count in test set:', sum(len(sent) for sent in test_data))

Построим маппинги из слов в индекс и из тега в индекс:


In [0]:
words = {word for sample in train_data for word, tag in sample}
word2ind = {word: ind + 1 for ind, word in enumerate(words)}
word2ind['<pad>'] = 0

tags = {tag for sample in train_data for word, tag in sample}
tag2ind = {tag: ind + 1 for ind, tag in enumerate(tags)}
tag2ind['<pad>'] = 0

print('Unique words in train = {}. Tags = {}'.format(len(word2ind), tags))

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter

tag_distribution = Counter(tag for sample in train_data for _, tag in sample)
tag_distribution = [tag_distribution[tag] for tag in tags]

plt.figure(figsize=(10, 5))

bar_width = 0.35
plt.bar(np.arange(len(tags)), tag_distribution, bar_width, align='center', alpha=0.5)
plt.xticks(np.arange(len(tags)), tags)
    
plt.show()

## Бейзлайн

Какой самый простой теггер можно придумать? Давайте просто запоминать, какие теги самые вероятные для слова (или для последовательности):

![tag-context](https://www.nltk.org/images/tag-context.png =x150)  
*From [Categorizing and Tagging Words, nltk](https://www.nltk.org/book/ch05.html)*

На картинке показано, что для предсказания $t_n$ используются два предыдущих предсказанных тега + текущее слово. По корпусу считаются вероятность для $P(t_n| w_n, t_{n-1}, t_{n-2})$, выбирается тег с максимальной вероятностью.

Более аккуратно такая идея реализована в Hidden Markov Models: по тренировочному корпусу вычисляются вероятности $P(w_n| t_n), P(t_n|t_{n-1}, t_{n-2})$ и максимизируется их произведение.

Простейший вариант - униграммная модель, учитывающая только слово:

In [0]:
import nltk

default_tagger = nltk.DefaultTagger('NN')

unigram_tagger = nltk.UnigramTagger(train_data, backoff=default_tagger)
print('Accuracy of unigram tagger = {:.2%}'.format(unigram_tagger.evaluate(test_data)))

Добавим вероятности переходов:

In [0]:
bigram_tagger = nltk.BigramTagger(train_data, backoff=unigram_tagger)
print('Accuracy of bigram tagger = {:.2%}'.format(bigram_tagger.evaluate(test_data)))

Обратите внимание, что `backoff` важен:

In [0]:
trigram_tagger = nltk.TrigramTagger(train_data)
print('Accuracy of trigram tagger = {:.2%}'.format(trigram_tagger.evaluate(test_data)))

## Увеличиваем контекст с рекуррентными сетями

Униграмная модель работает на удивление хорошо, но мы же собрались учить сеточки.

Омонимия - основная причина, почему униграмная модель плоха:
*“he cashed a check at the **bank**”*  
vs  
*“he sat on the **bank** of the river”*

Поэтому нам очень полезно учитывать контекст при предсказании тега.

Воспользуемся LSTM - он умеет работать с контекстом очень даже хорошо:

![](https://image.ibb.co/kgmoff/Baseline-Tagger.png =x400)

Синим показано выделение фичей из слова, LSTM оранжевенький - он строит эмбеддинги слов с учетом контекста, а дальше зелененькая логистическая регрессия делает предсказания тегов.

In [0]:
def convert_data(data, word2ind, tag2ind):
    X = [[word2ind.get(word, 0) for word, _ in sample] for sample in data]
    y = [[tag2ind[tag] for _, tag in sample] for sample in data]
    
    return X, y

X_train, y_train = convert_data(train_data, word2ind, tag2ind)
X_val, y_val = convert_data(val_data, word2ind, tag2ind)
X_test, y_test = convert_data(test_data, word2ind, tag2ind)

In [0]:
def iterate_batches(data, batch_size):
    X, y = data
    n_samples = len(X)

    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        
        batch_indices = indices[start:end]
        
        max_sent_len = max(len(X[ind]) for ind in batch_indices)
        X_batch = np.zeros((max_sent_len, len(batch_indices)))
        y_batch = np.zeros((max_sent_len, len(batch_indices)))
        
        for batch_ind, sample_ind in enumerate(batch_indices):
            X_batch[:len(X[sample_ind]), batch_ind] = X[sample_ind]
            y_batch[:len(y[sample_ind]), batch_ind] = y[sample_ind]
            
        yield X_batch, y_batch

In [0]:
X_batch, y_batch = next(iterate_batches((X_train, y_train), 4))

X_batch, y_batch

**Задание** Реализуйте `LSTMTagger`:

In [0]:
class LSTMTagger(nn.Module):
    def __init__(self, vocab_size, tagset_size, word_emb_dim=100, lstm_hidden_dim=128, lstm_layers_count=1):
        super().__init__()
        
        <create layers>

    def forward(self, inputs):
        <apply them>

**Задание** Научитесь считать accuracy и loss (а заодно проверьте, что модель работает)

In [0]:
model = LSTMTagger(
    vocab_size=len(word2ind),
    tagset_size=len(tag2ind)
)

X_batch, y_batch = torch.LongTensor(X_batch), torch.LongTensor(y_batch)

logits = model(X_batch)

<calc accuracy>

In [0]:
criterion = nn.CrossEntropyLoss()
<calc loss>

**Задание** Вставьте эти вычисление в функцию:

In [0]:
import math
from tqdm import tqdm


def do_epoch(model, criterion, data, batch_size, optimizer=None, name=None):
    epoch_loss = 0
    correct_count = 0
    sum_count = 0
    
    is_train = not optimizer is None
    name = name or ''
    model.train(is_train)
    
    batches_count = math.ceil(len(data[0]) / batch_size)
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=batches_count) as progress_bar:
            for i, (X_batch, y_batch) in enumerate(iterate_batches(data, batch_size)):
                X_batch, y_batch = LongTensor(X_batch), LongTensor(y_batch)
                logits = model(X_batch)

                loss = <calc loss>

                epoch_loss += loss.item()

                if optimizer:
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                cur_correct_count, cur_sum_count = <calc accuracy>

                correct_count += cur_correct_count
                sum_count += cur_sum_count

                progress_bar.update()
                progress_bar.set_description('{:>5s} Loss = {:.5f}, Accuracy = {:.2%}'.format(
                    name, loss.item(), cur_correct_count / cur_sum_count)
                )
                
            progress_bar.set_description('{:>5s} Loss = {:.5f}, Accuracy = {:.2%}'.format(
                name, epoch_loss / batches_count, correct_count / sum_count)
            )

    return epoch_loss / batches_count, correct_count / sum_count


def fit(model, criterion, optimizer, train_data, epochs_count=1, batch_size=32,
        val_data=None, val_batch_size=None):
        
    if not val_data is None and val_batch_size is None:
        val_batch_size = batch_size
        
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        train_loss, train_acc = do_epoch(model, criterion, train_data, batch_size, optimizer, name_prefix + 'Train:')
        
        if not val_data is None:
            val_loss, val_acc = do_epoch(model, criterion, val_data, val_batch_size, None, name_prefix + '  Val:')

In [0]:
model = LSTMTagger(
    vocab_size=len(word2ind),
    tagset_size=len(tag2ind)
).cuda()

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(model.parameters())

fit(model, criterion, optimizer, train_data=(X_train, y_train), epochs_count=50,
    batch_size=64, val_data=(X_val, y_val), val_batch_size=512)

### Masking

**Задание** Проверьте себя - не считаете ли вы потери и accuracy на паддингах - очень легко получить высокое качество за счет этого.

У функции потерь есть параметр `ignore_index`, для таких целей. Для accuracy нужно использовать маскинг - умножение на маску из нулей и единиц, где нули на позициях паддингов (а потом усреднение по ненулевым позициям в маске).

**Задание** Посчитайте качество модели на тесте

### Bidirectional LSTM

Благодаря BiLSTM можно использовать сразу оба контеста при предсказании тега слова. Т.е. для каждого токена $w_i$ forward LSTM будет выдавать представление $\mathbf{f_i} \sim (w_1, \ldots, w_i)$ - построенное по всему левому контексту - и $\mathbf{b_i} \sim (w_n, \ldots, w_i)$ - представление правого контекста. Их конкатенация автоматически захватит весь доступный контекст слова: $\mathbf{h_i} = [\mathbf{f_i}, \mathbf{b_i}] \sim (w_1, \ldots, w_n)$.

![BiLSTM](https://www.researchgate.net/profile/Wang_Ling/publication/280912217/figure/fig2/AS:391505383575555@1470353565299/Illustration-of-our-neural-network-for-POS-tagging.png =x450)  
*From [Finding Function in Form: Compositional Character Models for Open Vocabulary Word Representation](https://arxiv.org/abs/1508.02096)*

**Задание** Добавьте Bidirectional LSTM.

### Предобученные эмбеддинги

Мы знаем, какая клёвая вещь - предобученные эмбеддинги. При текущем размере обучающей выборки еще можно было учить их и с нуля - с меньшей было бы совсем плохо.

Поэтому стандартный пайплайн - скачать эмбеддинги, засунуть их в сеточку. Запустим его:

In [0]:
import gensim.downloader as api

w2v_model = api.load('glove-wiki-gigaword-100')

Построим подматрицу для слов из нашей тренировочной выборки:

In [0]:
known_count = 0
embeddings = np.zeros((len(word2ind), w2v_model.vectors.shape[1]))
for word, ind in word2ind.items():
    word = word.lower()
    if word in w2v_model.vocab:
        embeddings[ind] = w2v_model.get_vector(word)
        known_count += 1
        
print('Know {} out of {} word embeddings'.format(known_count, len(word2ind)))

**Задание** Сделайте модель с предобученной матрицей. Используйте `nn.Embedding.from_pretrained`.

In [0]:
class LSTMTaggerWithPretrainedEmbs(nn.Module):
    def __init__(self, embeddings, tagset_size, lstm_hidden_dim=64, lstm_layers_count=1):
        super().__init__()
        
        <create me>

    def forward(self, inputs):
        <use me>

In [0]:
model = LSTMTaggerWithPretrainedEmbs(
    embeddings=embeddings,
    tagset_size=len(tag2ind)
).cuda()

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters())

fit(model, criterion, optimizer, train_data=(X_train, y_train), epochs_count=50,
    batch_size=64, val_data=(X_val, y_val), val_batch_size=512)

**Задание** Оцените качество модели на тестовой выборке. Обратите внимание, вовсе не обязательно ограничиваться векторами из урезанной матрицы - вполне могут найтись слова в тесте, которых не было в трейне и для которых есть эмбеддинги.

In [0]:
<calc test accuracy>

### Дообучение предобученных векторов

**Задание** Почему бы не попробовать дообучать вектора? Для этого нужно просто заменить флаг `freeze=False` в методе `from_pretrained`. Попробуйте.

**Задание** На самом деле, понятно, почему это плохо - после этого нельзя использовать старые предобученные вектора (которые не попали в трейн). Проверьте, какое качество получается на тесте со старыми векторами.

Чтобы бороться с этим, можно использовать такой прием: на предобученные вектора накладывать $l_2$-регуляризацию, чтобы они не удалялись от исходных векторов, а для слов, эмбеддинги которых мы не знаем, строить случайные вектора и учить их как обычно.

Почитать про это можно чуть-чуть здесь: [Pseudo-rehearsal: A simple solution to catastrophic forgetting for NLP](https://explosion.ai/blog/pseudo-rehearsal-catastrophic-forgetting) либо в книжке Goldberg'а.

**Задание** Попробуйте реализовать это.

## We need to go deeper, сети символьного уровня

Напомню, на прошлом занятии мы строили LSTM сеть, которая обрабатывала последовательности символов, и предсказывала, к какому языку относится слово. 

LSTM выступал в роли feature extractor'а, работающего с произвольного размера последовательностью символов (ну, почти произвольного - мы ограничивались максимальной длиной слова). Батч для сети имел размерность `(max_word_len, batch_size)`.

Теперь мы опять хотим использовать такую же идею для извлечения признаков из последовательности символов - потому что последовательность символов же должна быть полезной для предсказания части речи, правда?

Сеть должна будет запомнить, например, что `-ly` - это часто про наречие, а `-tion` - про существительное.

![](https://image.ibb.co/kzbh6L/Char-Bi-LSTM.png =x400)

Остальная часть сети при этом будет такой же.

Найдем границу для длины слов:

In [0]:
from collections import Counter 
    
def find_max_len(counter, threshold):
    sum_count = sum(counter.values())
    cum_count = 0
    for i in range(max(counter)):
        cum_count += counter[i]
        if cum_count > sum_count * threshold:
            return i
    return max(counter)

word_len_counter = Counter()
for sent in data:
    for word, _ in sent:
        word_len_counter[len(word)] += 1
    
threshold = 0.99
MAX_WORD_LEN = find_max_len(word_len_counter, threshold)

print('Max word len for {:.0%} of words is {}'.format(threshold, MAX_WORD_LEN))

Построим алфавит:

In [0]:
from string import punctuation

def get_range(first_symb, last_symb):
    return set(chr(c) for c in range(ord(first_symb), ord(last_symb) + 1))

chars = get_range('a', 'z') | get_range('A', 'Z') | get_range('0', '9') | set(punctuation)
char2ind = {c : i + 1 for i, c in enumerate(chars)}
char2ind['<pad>'] = 0

**Задание** Сконвертируйте данные, как в функции выше - только теперь слова должны отобразиться не в один индекс, а в последовательность.

Обрезайте слова по `MAX_WORD_LEN`.

In [0]:
def convert_data(data, char2ind, tag2ind):
    X, y = <calc it>
    return X, y
  
X_train, y_train = convert_data(train_data, char2ind, tag2ind)
X_val, y_val = convert_data(val_data, char2ind, tag2ind)
X_test, y_test = convert_data(test_data, char2ind, tag2ind)

Напишем генератор батчей:

In [0]:
def iterate_batches(data, batch_size):
    X, y = data
    n_samples = len(X)

    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    for start in range(0, n_samples, batch_size):
        end = min(start + batch_size, n_samples)
        
        batch_indices = indices[start: end]
        
        sent_len = max(len(X[ind]) for ind in batch_indices)
        word_len = max(len(word) for ind in batch_indices for word in X[ind])
            
        X_batch = np.zeros((sent_len, len(batch_indices), word_len))
        y_batch = np.zeros((sent_len, len(batch_indices)))
        
        for batch_ind, sample_ind in enumerate(batch_indices):
            for word_ind, word in enumerate(X[sample_ind]):
                X_batch[word_ind, batch_ind, :len(word)] = word
            y_batch[:len(y[sample_ind]), batch_ind] = y[sample_ind]
            
        yield X_batch, y_batch

In [0]:
X_batch, y_batch = next(iterate_batches((X_train, y_train), 4))

X_batch.shape, y_batch.shape

**Задание** Реализуйте сеть, которая принимает батч размера `(seq_len, batch_size, word_len)` и возвращает `(seq_len, batch_size, word_emb_dim)`. Это может быть любая функция, которая умеет в последовательности произвольной длины. Мы уже смотрели на сверточные и рекуррентные сети для такой задачи - попробуйте обе.

In [0]:
class CharsEmbedding(nn.Module):
    def __init__(self, vocab_size, char_emb_dim=24, word_emb_dim=100):
        super().__init__()
        
        <create Conv or LSTM encoder>
        
    def forward(self, inputs):
        <apply>

**Задание** Реализуйте теггер с эмбеддингами символьного уровня.

In [0]:
class LSTMTagger(nn.Module):
    def __init__(self, char_vocab_size, tagset_size, char_emb_dim=24, 
                 word_emb_dim=128, lstm_hidden_dim=128, lstm_layers_count=1):
        super().__init__()
        
        <create it>

    def forward(self, inputs):
        <apply>

In [0]:
model = LSTMTagger(char_vocab_size=len(char2ind), tagset_size=len(tag2ind)).cuda()

criterion = nn.CrossEntropyLoss(ignore_index=0).cuda()
optimizer = optim.Adam(model.parameters())

fit(model, criterion, optimizer, train_data=(X_train, y_train), epochs_count=20, 
    batch_size=24, val_data=(X_val, y_val), val_batch_size=32)

**Задание** Оцените его качество.

In [0]:
_, test_accuracy = do_epoch(model, criterion, (X_test, y_test), batch_size=32)

### Визуализации

**Задание** Посчитайте эмбеддинги символьного уровня (обученные внутри модели перед этим) для 1000 случайных слов из `word2ind`.

In [0]:
embeddings, index2word = <calc me>

In [0]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook

from sklearn.manifold import TSNE
from sklearn.preprocessing import scale


def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    output_notebook()
    
    if isinstance(color, str): 
        color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: 
        pl.show(fig)
    return fig


def get_tsne_projection(word_vectors):
    tsne = TSNE(n_components=2, verbose=100)
    return scale(tsne.fit_transform(word_vectors))
    
    
def visualize_embeddings(embeddings, token):
    tsne = get_tsne_projection(embeddings)
    draw_vectors(tsne[:, 0], tsne[:, 1], token=token)
    

visualize_embeddings(embeddings, index2word)

**Задание** Посчитайте эмбеддинги для всех слов из трейна и для нескольких случайных слов из теста, которые не встречаются в трейне, найдите их ближайших соседей по их эмбеддигам символьного уровня.

### Словные эмбеддинги

**Задание** Только символьных эмбеддингов может быть недостаточно. Верните ещё словные эмбеддинги. Слова стоит приводить к нижнему регистру - признаки, связанные с регистром должны ухватываться символьный LSTM.

Эти эмбеддинги можно просто сконкатенировать, можно складывать, а можно использовать гейт (как в LSTM). Например, по эмбеддингу слова предсказывать $o = \sigma(w)$ - насколько он хорош и сочетать в такой пропорции с символьным эмбеддингом: $o \odot w + (1 - o) \odot \tilde w$, где $\tilde w$ - эмбеддинг слова, полученный по символьному уровню. Проверьте разные варианты.

### Связь словных эмбеддингов и эмбеддингов символьного уровня
В словных эмбеддингах мы строим отображение из слова в индекс. В итоге входной батч достаточно небольшой - это хорошо для обучения (быстрее передача на видеокарту). С символьными эмбеддингами беда - но это можно исправить.

Давайте предпосчитаем для каждого слова в `word2ind` его последовательность индексов символов. Получится матрица. Эту матрицу можно вместе с моделью перенести на видеокарту. Тогда нужен будет батч из индексов слов - по нему можно сделать лукап (с помощью `F.embedding`) в матрице и получить трехмерную матрицу с символами.

Преимущество - по одному батчу можно получить сразу и эмбеддинги слов, и эмбеддинги символьного уровня. Это удобно и энергоэффективно.

Другая идея - после того, как мы обучили модель, можно предпосчитать эмбеддинги слов символьного уровня - лукап в таблице эмбеддингов гораздо проще, чем сверточная или рекуррентная сеть над символами. Таким образом, например, получаются эмбеддинги в [FastText](https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md) - они также исходно считаются на символьном (N-граммном) уровне.

## Encoder-decoder

Можно усложнить модель - добавить еще один рекуррентный слой. Первый слой будет служить энкодером последовательности, второй, более легкий - декодировать последовательность. Декодировать - значит, на вход он должен принимать как состояние для данного токена из энкодера, так и предыдущий предсказанный тег.

Выглядеть всё будет как-то так:

![encoder-decoder](https://image.ibb.co/jOrfT0/Encoder-Decoder.png =x600)

Зеленое - уже `LSTM`, а не `Linear`, а принимает оно сразу скрытое состояние от предыдущего токена (зеленая стрелка), предыдущий предсказанный тег (пунктирная стрелка) и состояние из BiLSTM - контекстное представление слова.

Тренироваться данная модель должна с teacher-forcing - передачей правильных меток в качестве ответов по пунктирным стрелкам. На предсказании же нужно реализовать beam search - держать сразу несколько лучших путей (последовательностей тегов) для декодируемой последовательности.

**Задание** Рискните реализовать это.

(А вообще мы будем разбираться с этим подробнее, когда дойдем до машинного перевода - можно вернуться сюда после него :) )

# Дополнительные материалы

## Классические подходы
Speech and Language Processing, Chapter 8, Part-of-speech Tagging. Daniel Jurafsky [[pdf](https://web.stanford.edu/~jurafsky/slp3/8.pdf)]

## Статьи
Learning Character-level Representations for Part-of-Speech Tagging, dos Santos et al, 2014 [pdf](http://proceedings.mlr.press/v32/santos14.pdf)  
Finding Function in Form: Compositional Character Models for Open Vocabulary Word Representation, Wang Ling et al, 2015 [arxiv](https://arxiv.org/abs/1508.02096)  
Bidirectional LSTM-CRF Models for Sequence Tagging, Zhiheng Huang et al, 2015 [arxiv](https://arxiv.org/abs/1508.01991)  
End-to-end Sequence Labeling via Bi-directional LSTM-CNNs-CRF, Xuezhe Ma et al, 2016 [arxiv](https://arxiv.org/abs/1603.01354)  
Improving Part-of-speech Tagging via Multi-task Learning and Character-level Word Representations, Daniil Anastasyev et al, 2018 [pdf](http://www.dialog-21.ru/media/4282/anastasyevdg.pdf) :)  

# Сдача

[Опрос](https://goo.gl/forms/R6UqcESWIjtVSA6J3)